# The Analysis of several Binary Star Configurations to find the B-Spline Signal

In [1]:
%load_ext autoreload
%autoreload 2
# to ensure kernel resets when files change around it

In [2]:
import numpy as np # main library for numeric calculations
import pandas as pd # main library for data analysis
import matplotlib.pyplot as plt # main library for data plotting
import seaborn as sns # Another library for data plotting with more functions
sns.set()

import scipy as scp # STEM software
from scipy import signal
import scipy.interpolate as interpolate

from glob import glob # check files
from IPython.display import display, Markdown

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

import lightkurve as lk
import astropy

import sys, os

file_path = os.getcwd()
print(file_path)
parent_dir = "\\".join(file_path.split("\\")[:-1])
print(parent_dir)

sys.path.append(parent_dir)

from kepler import *

"""
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)"""

import warnings
warnings.filterwarnings('ignore') # literally to ignore all the warnings that don't matter


def printf(*args, sep=" ", end="\n"):
    string = sep.join([str(i) for i in args])+end
    display(Markdown(string))


c:\users\prannaya\appdata\local\programs\python\python37\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


C:\Users\Prannaya\ThreeBody\notebooks
C:\Users\Prannaya\ThreeBody


In [3]:
# Enabling the `widget` backend.
# This requires jupyter-matplotlib a.k.a. ipympl.
# ipympl can be install via pip or conda.
!pip install ipympl
%matplotlib widget

# Testing matplotlib interactions with a simple plot
fig = plt.figure()
fig.canvas.header_visible = False # Hide the Figure name at the top of the figure
fig.canvas.toolbar_visible = True

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
binarystars = pd.read_csv("../data/binarystars.csv").rename(columns={"KIC": "id"}).set_index("id")
configs = pd.read_csv("../data/binaryconfigs.csv")
configs.columns = ["id", "period", "duration", "depth"]
configs = configs.set_index("id")

### Kepler Eclipsing Binary Catalog

In [5]:
binarystars

,period,period_err,bjd0,bjd0_err,morph,GLon,GLat,kmag,Teff,SC
id,,,,,,,,,,
3863594,0.053268,0.0,55000.000000,0.004327,0.79,-1.0000,-1.0000,-1.000,-1.0,False
10417986,0.073731,0.0,55000.027476,0.004231,0.99,81.0390,11.0820,9.128,-1.0,True
8912468,0.094838,0.0,54953.576945,0.005326,0.98,80.1095,7.8882,11.751,6194.0,False
8758716,0.107205,0.0,54953.672989,0.006197,1.00,77.7478,11.6565,13.531,-1.0,False
10855535,0.112782,0.0,54964.629315,0.006374,0.99,79.3949,15.9212,13.870,7555.0,False
...,...,...,...,...,...,...,...,...,...,...
9408440,989.985000,-1.0,55346.365980,0.096130,0.00,78.5607,12.2615,13.199,5688.0,False
8054233,1058.000000,-1.0,54751.806288,0.968052,0.03,78.6142,7.7321,11.783,4733.0,False
7672940,1064.270000,-1.0,54977.092960,0.089646,0.00,74.5296,14.6136,12.328,-1.0,False


In [6]:
def day2seconds(s):
    if type(s) in [float, int]:
        return s * 86400
    return float(s[:-2]) * 86400

configs["period"] = configs.period.apply(day2seconds)
configs["duration"] = configs.duration.apply(day2seconds)
configs = configs.dropna().sort_index()
configs

,period,duration,depth
id,,,
1026032,7.303493e+05,17280.0,0.003519
1026957,1.127302e+06,28512.0,0.001687
1161345,3.707062e+05,4320.0,0.000869
1295531,7.281519e+04,28512.0,0.007980
1571511,3.140644e+04,28512.0,0.000048
...,...,...,...
12598713,3.332850e+04,28512.0,0.011091
12599700,8.804085e+04,28512.0,0.038670
12644769,1.055417e+06,8640.0,0.007281


In [31]:
binarystars.to_csv("../data/kepler_binary_stars.csv")
configs.to_csv("../data/kepler_binary_configs.csv")

In [40]:
altogether = pd.merge(configs, binarystars, how="left", on="id")
altogether.to_csv("../data/kepler_binary.csv")

## EDA
Exploratory Data Analysis

### Constants

In [7]:
n = len(configs)
bins = int(np.sqrt(n))

periods = configs.period
durations = configs.duration
depths = configs.depth

percentiles = np.array([2.5, 25, 50, 75, 97.5])

### Summary Statistics

In [41]:
mu_period = periods.mean()
sigma_period = periods.std()
var_period = np.var(periods)
ptiles_per = np.percentile(periods, percentiles)

mu_duration = durations.mean()
sigma_duration = durations.std()
var_duration = np.var(durations)
ptiles_dur = np.percentile(durations, percentiles)

mu_depth = depths.mean()
sigma_duration = depths.std()
var_depth = np.var(depths)
ptiles_dep = np.percentile(depths, percentiles)

### Functions

In [9]:
def ecdf(data):
    """Compute ECDF for a one-dimensional array of measurements."""
    # Number of data points: n
    n = len(data)

    # x-data for the ECDF: x
    x = np.sort(data)

    # y-data for the ECDF: y
    y = np.arange(1, n+1) / n

    return x, y

def plot_hist(data, xlabel, ylabel="count", bins=bins, ax=None, **kwargs):
    if not ax:
        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111)
    
    ax.hist(data, bins=bins, **kwargs)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    plt.show()

    
def plot_ecdf(data, xlabel, ptiles, ylabel="ECDF", ax=None, marker=".", linestyle="none", **kwargs):
    if not ax:
        fig = plt.figure(figsize=(16, 12))
        ax = fig.add_subplot(111)
        
    # Retrive ECDF
    x, y = ecdf(data)

    # Generate plot
    ax.plot(x, y, marker=marker, linestyle=linestyle, **kwargs)

    # Label the axes
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    
    ax.plot(ptiles, percentiles/100, marker='D', color='red',
         linestyle="none")

    # Display the plot
    plt.show()

### Graphical EDA
#### Histograms

In [43]:
plot_hist(periods, "Period")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plot_hist(durations, "Duration")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
plot_hist(depths, "Depth")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### ECDF

In [13]:
plot_ecdf(periods, "Period", ptiles_per)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
plot_ecdf(durations, "Duration", ptiles_dur)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
plot_ecdf(depths, "Depth", ptiles_dep)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Quantitative EDA

#### Covariance

In [16]:
cov_perdur = np.cov(periods, durations)
cov_perdep = np.cov(periods, depths)
cov_durdep = np.cov(durations, depths)

#### Correlation

In [17]:
configs.corr()

,period,duration,depth
period,1.000000,-0.288703,-0.099790
duration,-0.288703,1.000000,-0.173623
depth,-0.099790,-0.173623,1.000000


In [18]:
plt.figure(figsize=(16, 12))
heatmap = sns.heatmap(configs.corr(), vmin=-1, vmax=1, annot=True)
heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':18}, pad=12)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Pearson Coefficient

In [19]:
def pearson_r(x, y):
    """Compute Pearson correlation coefficient between two arrays."""
    corr_mat = np.corrcoef(x, y)
    return corr_mat[0,1]

pearson_perdur = pearson_r(periods, durations)
pearson_perdep = pearson_r(periods, depths)
pearson_durdep = pearson_r(durations, depths)

## Signal Processing

In [20]:
def evaluateAndProcessData(x, y):
    t, c, k = interpolate.splrep(x, y, s=0, k=4)
    xx = np.linspace(x.min(), x.max(), 100)
    spline = interpolate.BSpline(t, c, k, extrapolate=False)
    
    plt.figure(figsize=(16,6))
    plt.plot(x, y, 'bo', label='Original points')
    plt.plot(x, y)
    plt.plot(xx, spline(xx), 'r', label='BSpline')
    plt.grid()
    plt.legend(loc='best')
    plt.show()
    
# Retrieving all the important details
def getData(id):
    try:
        return retrieveKeplerLightCurve(id)
    except:
        return None


In [29]:
ids = list(configs.index)
id2 = np.random.choice(ids)
id2

4350454

## Analysis of KIC 12055255

In [22]:
lc = getData(id2)
lc

time,flux,flux_err,quality,timecorr,centroid_col,centroid_row,cadenceno,sap_flux,sap_flux_err,sap_bkg,sap_bkg_err,pdcsap_flux,pdcsap_flux_err,sap_quality,psf_centr1,psf_centr1_err,psf_centr2,psf_centr2_err,mom_centr1,mom_centr1_err,mom_centr2,mom_centr2_err,pos_corr1,pos_corr2
,,,,d,pix,pix,,electron / s,electron / s,electron / s,electron / s,electron / s,electron / s,,pix,pix,pix,pix,pix,pix,pix,pix,pix,pix
object,float32,float32,int32,float32,float64,float64,int32,float32,float32,float32,float32,float32,float32,int32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32
131.51219531564857,9.5549995e-01,8.5038057e-04,0,1.201496e-03,85.59066,708.31774,1105,7.5554502e+03,4.5662069e+00,1.6199392e+03,7.3718590e-01,6.0997803e+03,5.4287124e+00,0,nan,nan,nan,nan,85.59066,9.1063807e-04,708.31774,9.6375920e-04,-1.0769945e-02,2.8466804e-02
131.5326296267085,9.1290057e-01,8.4968633e-04,0,1.202107e-03,85.59039,708.31152,1106,7.4945859e+03,4.5622525e+00,1.6163114e+03,7.3891795e-01,5.8278315e+03,5.4242806e+00,0,nan,nan,nan,nan,85.59039,9.1825414e-04,708.31152,9.7315677e-04,-1.0800323e-02,2.8523881e-02
131.55306383777497,9.5164818e-01,8.5255754e-04,0,1.202718e-03,85.59732,708.32186,1107,7.7014619e+03,4.5777564e+00,1.6162489e+03,7.3888350e-01,6.0751909e+03,5.4426098e+00,0,nan,nan,nan,nan,85.59732,8.9510053e-04,708.32186,9.4749569e-04,-1.0927333e-02,2.8716728e-02
131.57349794871698,1.0011353e+00,8.5619063e-04,0,1.203329e-03,85.60837,708.33262,1108,7.9500029e+03,4.5984235e+00,1.6191105e+03,7.3777473e-01,6.3911104e+03,5.4658031e+00,0,nan,nan,nan,nan,85.60837,8.7010994e-04,708.33262,9.2017808e-04,-1.0860695e-02,2.8451622e-02
131.59393215966702,1.0095477e+00,8.5672532e-04,0,1.203940e-03,85.60750,708.33273,1109,7.9784233e+03,4.6001711e+00,1.6163134e+03,7.3863536e-01,6.4448135e+03,5.4692163e+00,0,nan,nan,nan,nan,85.60750,8.6735451e-04,708.33273,9.1718580e-04,-1.0759398e-02,2.8272571e-02
131.61436647061782,9.7349161e-01,8.5402047e-04,10000000000000,1.204551e-03,85.60018,708.32474,1110,7.7549326e+03,4.5826530e+00,1.6172587e+03,7.3834467e-01,6.2146362e+03,5.4519491e+00,10000000000000,nan,nan,nan,nan,85.60018,8.8958058e-04,708.32474,9.4137952e-04,-1.0664177e-02,2.8054871e-02
131.6348005814507,9.4427717e-01,8.5081661e-04,10000010000000,1.205161e-03,85.59344,708.31441,1111,7.5538955e+03,4.5668297e+00,1.6193079e+03,7.3761410e-01,6.0281353e+03,5.4314961e+00,10000010000000,nan,nan,nan,nan,85.59344,9.1181020e-04,708.31441,9.6608087e-04,-1.0520628e-02,2.7945507e-02
131.65523479216063,9.6751630e-01,8.5185817e-04,0,1.205772e-03,85.59510,708.31704,1112,7.6254341e+03,4.5722513e+00,1.6188052e+03,7.3782492e-01,6.1764907e+03,5.4381452e+00,0,nan,nan,nan,nan,85.59510,9.0381951e-04,708.31704,9.5738674e-04,-1.0358448e-02,2.7873931e-02


In [23]:
plotKeplerLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
plotKeplerSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
plotKeplerPDCSAPLightCurve(lc)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [26]:
time = np.array(list(map(lambda time: time.value, list(lc.time))))
time

array([ 131.51219532,  131.53262963,  131.55306384, ..., 1590.96068149,
       1590.98111561, 1591.00154992])

In [27]:
flux = np.array(list(map(lambda flux: flux.value, list(lc.flux))))
flux

array([0.95549995, 0.91290057, 0.9516482 , ..., 0.9792476 , 1.0175403 ,
       1.0432092 ], dtype=float32)

In [28]:
evaluateAndProcessData(time, flux)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …